# S&P 500 P/E Analysis - Regression Modeling - Model Selection Using L1  

### Credit:
    Data for this exercise is based on the S&P 500 companies fundamental data provided by Dominik Gawlik at
    https://www.kaggle.com/dgawlik/nyse

## Learning Objectives

Run a basic regression using L1 regularization to conduct variable selection
    * run a simple OLS
    * analyze output
    * run L1 regression
    * interpret model parameters

## Imports

In [1]:
import pandas as pd
import statsmodels.api as sms
import sklearn.linear_model as lm

## Get Data and Subset Data

In [2]:
# Import data from the csv file
df = pd.read_csv('data/relative_valuation.csv')

In [3]:
# subset the dataframe removing rows with NULL values
bix = df.notnull().all(axis=1)
df = df[bix]

## A First Regression Model Using All Variables

In [4]:
df.describe()

,EarningsPerShare,Pre-TaxROE,AfterTaxROE,CashRatio,QuickRatio,GrossMargin,OperatingMargin,Pre-TaxMargin,debt_to_asset,debt_to_equity,operating_cash_flow_margin,profit_margin,ebit_margin,p/e,p/s,capital_surplus_to_asset,Goodwill_to_asset
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,3.458767,22.294521,15.424658,47.726027,123.602740,30.479452,12.082192,11.109589,0.227528,0.750862,13.691306,8.313016,12.753143,21.888464,1.400077,16.900353,17.737734
std,2.378205,10.204913,6.803484,55.610780,81.785209,10.876244,6.039533,6.568602,0.125635,0.704992,8.638832,4.787935,7.364707,12.497488,3.174792,17.061349,16.369561
min,-2.580000,1.000000,2.000000,0.000000,10.000000,6.000000,0.000000,1.000000,0.000000,0.000000,-9.241430,-2.252885,1.409390,-28.779070,0.000000,0.000000,0.000000
25%,1.860000,15.000000,10.250000,11.250000,71.000000,23.000000,8.000000,6.250000,0.144506,0.297018,7.498277,4.645786,7.306614,15.544375,0.015095,4.341478,1.372582
50%,2.970000,22.500000,15.000000,31.000000,104.500000,32.000000,12.000000,11.000000,0.228223,0.604959,12.497273,7.820948,12.338856,19.937240,0.051733,11.979672,14.443672
75%,4.415000,30.000000,20.750000,60.000000,154.750000,39.000000,16.750000,15.000000,0.309276,0.934102,18.348204,11.074523,16.729717,24.846098,1.104627,25.109497,30.055251
max,14.350000,45.000000,29.000000,309.000000,619.000000,49.000000,28.000000,47.000000,0.627004,4.876463,48.471436,27.681027,56.710750,81.619043,18.767629,91.464171,61.195090


In [5]:
columns = [ 'Pre-TaxROE',
            'AfterTaxROE',
            'CashRatio', 
           'QuickRatio',
           'OperatingMargin',
           'Pre-TaxMargin', 
           'profit_margin',
           'operating_cash_flow_margin',
           'debt_to_equity', 
           'debt_to_asset', 
           'capital_surplus_to_asset',
           'Goodwill_to_asset',
          ]

#### Specify the OLS Model

In [6]:
model = sms.OLS(df['p/e'], df[columns])

#### Fit the model

In [7]:
result = model.fit()

Notes:

* Adjusted R-Square at 0.778 indicates a resonably good model fit

* Several variables are statistically significant
  * Pre-TaxROE
  * AfterTaxROE
  * OperatingMargin
  * Pre-TaxMargin
  * profit_margin
  * capital_surplus_to_asset
  * goodwill_to_asset


 * Warning Signs
  * Warning indicating strong multicollinearity
  * Regression coefficients defies common sense
    * AfterTaxROE coefficient (statistically significant) is negative (-2.5335) indicating a high valuation for firms with lower ROE
    * Pre-TarMargin coefficient (statistically significant) is negative (-2.4156) indicating a high valuation for firms with lower margin.
  
  
Large number of collinear variables.  How do we select ones that are useful yet uncorrelated?

#### Print the summary of the model results

In [8]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    p/e   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     43.67
Date:                Mon, 27 Mar 2017   Prob (F-statistic):           2.25e-40
Time:                        15:06:11   Log-Likelihood:                -562.01
No. Observations:                 146   AIC:                             1148.
Df Residuals:                     134   BIC:                             1184.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Pre-TaxROE          

### Variable selection using L1 regression

Notes:
    Good model should be less complex.
        * fit L1 penalized model  
        * we choose the model with the minimal bayes information criterion (bic)
        * this will force an increasing number of the parameters to zero
        * the nonzero variables are the selected variables
        * last we will refit the "best" model and interpret the variables

#### Specify L1 Regression Model

In [9]:
reg = lm.LassoLarsIC(normalize=True, criterion='bic')

#### Fit the L1 Regression Model

In [10]:
reg.fit(df[columns], df['p/e'])

LassoLarsIC(copy_X=True, criterion='bic', eps=2.2204460492503131e-16,
      fit_intercept=True, max_iter=500, normalize=True, positive=False,
      precompute='auto', verbose=False)

#### View the model coefficients

In [11]:
reg.coef_

array([ 0.        , -0.36885292,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.0952389 ,  0.        ,  0.        ,
        0.05578808,  0.14554785])

#### Select non-zero variables

Of the twelve variables, the selected four variables can explain the data just as good as all twelve combined.

In [12]:
subset_columns = [columns[i] for i in range(len(columns)) if abs(reg.coef_[i])>0]
subset_columns

['AfterTaxROE',
 'operating_cash_flow_margin',
 'capital_surplus_to_asset',
 'Goodwill_to_asset']

### Run regression on the "simple" model

#### Build a new regression model using the selected variables

In [13]:
model = sms.OLS(df['p/e'], df[subset_columns])

In [14]:
result = model.fit()      

Notes:
    * Adjusted R-Square at 0.705
    
    * The coefficient for AfterTaxROE is positive indicating that firms with higher ROE will be valued more than firms with lower ROE.  Note that this coefficient was negative in our "all-variable" model.
    
    * operating_cash_flow_margin which as insignificant in our previous model is significant now.
    
    
Interpretation:
    
    Keeping all things the same,
    * 1% increase in AfterTaxROE increases the P/E by 0.4343
    * 1% increase in operating cash flow margin increases the P/E by 0.1946
    * 1% increase in surplus capital relative to the asset increases the P/E by 0.2852
    * 1% increase in goodwill relative to the asset increases the P/E by 0.3120

In [15]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    p/e   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     88.13
Date:                Mon, 27 Mar 2017   Prob (F-statistic):           1.73e-37
Time:                        15:06:12   Log-Likelihood:                -587.10
No. Observations:                 146   AIC:                             1182.
Df Residuals:                     142   BIC:                             1194.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
AfterTaxROE         